## Course Assignment Instructions
You should have Python (version 3.8 or later) and Jupyter Notebook installed to complete this assignment. You will write code in the empty cell/cells below the problem. While most of this will be a programming assignment, some questions will ask you to "write a few sentences" in markdown cells. 

Submission Instructions:

Create a labs directory in your personal class repository (e.g., located in your home directory)
Clone the class repository
Copy this Jupyter notebook file (.ipynb) into your repo/labs directory
Make your edits, commit changes, and push to your repository
All submissions must be pushed before the due date to avoid late penalties. 

Labs are graded out of a 100 pts. Each day late is -10. For a max penalty of -50 after 5 days. From there you may submit the lab anytime before the semester ends for a max score of 50.  

Lab 5 is due on 3/17/25

Write a function spec'd as follows:

Provide predictions for each of these computations and then run them to make sure you're correct.

In [3]:
import numpy as np

def norm_vec(x):
    """Compute the Euclidean norm of a vector."""
    return np.sqrt(np.sum(x**2))

def orthogonal_projection(a, v):
    """
    Projects vector a onto vector v.
    """
    # Ensure numpy arrays
    a = np.asarray(a)
    v = np.asarray(v)
    
    # Compute the squared norm of v
    v_norm_sq = norm_vec(v)**2
    
    # Create the projection matrix using outer product
    H = np.outer(v, v) / v_norm_sq
    
    # Parallel and perpendicular components
    a_parallel = H @ a
    a_perpendicular = a - a_parallel
    
    return {'a_parallel': a_parallel, 'a_perpendicular': a_perpendicular}


In [5]:
# Test 1:
print("Test 1: orthogonal_projection([1,2,3,4], [1,2,3,4])")
result1 = orthogonal_projection(np.array([1,2,3,4]), np.array([1,2,3,4]))
print("a_parallel:")
print(result1['a_parallel'])        # Expected: [1, 2, 3, 4]
print("a_perpendicular:")
print(result1['a_perpendicular'])     # Expected: [0, 0, 0, 0]


# Test 2:
print("\nTest 2: orthogonal_projection([1,2,3,4], [0,2,0,-1])")
result2 = orthogonal_projection(np.array([1,2,3,4]), np.array([0,2,0,-1]))
print("a_parallel:")
print(result2['a_parallel'])          # Expected: [0, 0, 0, 0]
print("a_perpendicular:")
print(result2['a_perpendicular'])       # Expected: [1, 2, 3, 4]


# Test 3:
print("\nTest 3: result = orthogonal_projection([2,6,7,3], [1,3,5,7]*37)")

# Multiply [1,3,5,7] by 37
v3 = np.array([1,3,5,7]) * 37
result3 = orthogonal_projection(np.array([2,6,7,3]), v3)

# Compute dot product between a_parallel and a_perpendicular (should be near 0)
dot_product = np.dot(result3['a_parallel'], result3['a_perpendicular'])
print("Dot product (should be near 0):", dot_product)

# Sum of parallel and perpendicular components should return the original vector
sum_vector = result3['a_parallel'] + result3['a_perpendicular']
print("Sum (should equal original vector [2,6,7,3]):")
print(sum_vector)

# Compute the "percentage" of the projection relative to v3
percentage = result3['a_parallel'] / v3
print("a_parallel divided by ([1,3,5,7]*37) (percentage of projection):")
print(percentage)



Test 1: orthogonal_projection([1,2,3,4], [1,2,3,4])
a_parallel:
[1. 2. 3. 4.]
a_perpendicular:
[0. 0. 0. 0.]

Test 2: orthogonal_projection([1,2,3,4], [0,2,0,-1])
a_parallel:
[0. 0. 0. 0.]
a_perpendicular:
[1. 2. 3. 4.]

Test 3: result = orthogonal_projection([2,6,7,3], [1,3,5,7]*37)
Dot product (should be near 0): -3.552713678800501e-15
Sum (should equal original vector [2,6,7,3]):
[2. 6. 7. 3.]
a_parallel divided by ([1,3,5,7]*37) (percentage of projection):
[0.02445302 0.02445302 0.02445302 0.02445302]


Create a vector y by simulating n = 100 standard iid normals. Create a matrix of size 100 x 2 and populate the first column by all ones (for the intercept) and the second column by 100 standard iid normals. Find the R^2 of an OLS regression of `y ~ X`. Use matrix algebra.

In [8]:
import numpy as np

# Set seed for reproducibility
np.random.seed(123)

# Simulate data
n = 100
y = np.random.randn(n, 1)  # y as a (100 x 1) column vector
X = np.hstack((np.ones((n, 1)), np.random.randn(n, 1)))  # (100 x 2) design matrix

# Compute OLS coefficients: beta_hat = (X'X)^(-1) X'y
beta_hat = np.linalg.inv(X.T @ X) @ (X.T @ y)

# Compute the Hat matrix H = X (X'X)^(-1) X'
H = X @ np.linalg.inv(X.T @ X) @ X.T

# Identity matrix
I = np.eye(n)

# Compute SSR (Residual Sum of Squares): SSR = y' (I - H) y
SSR = (y.T @ (I - H) @ y).item()

# Centering matrix: M = I - (1/n) * 11'
M = I - (1/n) * np.ones((n, n))

# Compute SST (Total Sum of Squares): SST = y' M y
SST = (y.T @ M @ y).item()

# Compute R²
R2 = 1 - SSR / SST

# Print results
print("SSR:", SSR)
print("SST:", SST)
print("R²:", R2)


SSR: 127.2452490501258
SST: 127.29265592243308
R²: 0.00037242425310191063


Write a for loop to each time bind a new column of 100 standard iid normals to the matrix X and find the R^2 each time until the number of columns is 100. Create a vector to save all R^2's. What happened??

In [11]:
import numpy as np

# Set seed again for reproducibility
np.random.seed(123)

# Simulate response variable
n = 100
y = np.random.randn(n)  # y as a flat 1D array

# Start with intercept-only matrix (n x 1)
X = np.ones((n, 1))

# Pre-allocate list to save R² values
R2_list = []

# Loop: each time add a new column
for i in range(100):
    if i > 0:
        new_col = np.random.randn(n, 1)
        X = np.hstack((X, new_col))
    
    # Fit OLS using normal equation
    beta_hat = np.linalg.inv(X.T @ X) @ (X.T @ y)
    
    # Fitted values
    y_hat = X @ beta_hat
    
    # Calculate SST and SSR
    SST = np.sum((y - np.mean(y))**2)
    SSR = np.sum((y - y_hat)**2)
    
    # Calculate R²
    R2 = 1 - SSR / SST
    R2_list.append(R2)

# Print R² values
print("R² values for models with 1 to 100 columns:")
print(R2_list)


R² values for models with 1 to 100 columns:
[0.0, 0.00037242425310191063, 0.0039534712234703395, 0.015515917195883855, 0.016012956955489988, 0.060483757420479956, 0.0604856345047039, 0.06791713923101828, 0.1155205486782005, 0.115985431479941, 0.162531991451342, 0.16324852246361632, 0.1653058735950539, 0.16726405487373353, 0.16959752126125316, 0.186791654654901, 0.18680390934298918, 0.19280704730131826, 0.19656454956494696, 0.20676806135980885, 0.2067805910051016, 0.21297298468610137, 0.21413227261249568, 0.22344231007274185, 0.2644021621922207, 0.2688518991099367, 0.290450808990139, 0.29132140616382185, 0.2940661523691507, 0.29505341769528637, 0.3224612676383529, 0.34069205476632747, 0.34966372664455925, 0.36519299490256085, 0.36609673260436104, 0.38741665786176305, 0.403060815708544, 0.40637996512050323, 0.4084185281712275, 0.4086532629450571, 0.4087302243058517, 0.4122560226143148, 0.4199723297044975, 0.45296272233899937, 0.45435748259751674, 0.45838951359740276, 0.4584411626637773, 

What happened? As more predictors are added, even if they are just noise, the R2 generally increases, relfecting that the model is overfitting the data.

Test that the projection matrix onto this X is the same as I_n.

In [15]:
# Compute the projection matrix P = X (X'X)^(-1) X'
P = X @ np.linalg.inv(X.T @ X) @ X.T

# Define the identity matrix I_n
I_n = np.eye(n)

# Test if P is (numerically) equal to I_n
if np.allclose(P, I_n):
    print("Projection matrix P equals the identity matrix I_n.")
else:
    print("Projection matrix P does NOT equal the identity matrix I_n.")

# Optional: use an assertion for automatic checking
assert np.allclose(P, I_n), "Projection matrix does not equal identity matrix!"


Projection matrix P equals the identity matrix I_n.


Add one final column to X to bring the number of columns to 101. Then try to compute R^2. What happens? 

In [18]:
# Add one extra random column
extra_col = np.random.randn(n, 1)

# Bind it to X to make X_new (100 x 101)
X_new = np.hstack((X, extra_col))

# Try to compute beta_hat
try:
    beta_hat = np.linalg.inv(X_new.T @ X_new) @ (X_new.T @ y)
    y_hat = X_new @ beta_hat
    SST = np.sum((y - np.mean(y))**2)
    SSR = np.sum((y - y_hat)**2)
    R2 = 1 - SSR / SST
    print("R²:", R2)
except np.linalg.LinAlgError as e:
    print("Error in computing beta_hat:", e)
    print("Could not compute R² because X'X is singular.")


R²: 0.713290675962704


Why does this make sense?

You cannot fit a full model when p > n because there’s too much flexibility and not enough data to estimate all parameters.

Let's use the Boston Housing Data for the following exercises

In [23]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Load the Boston housing data
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep=r"\s+", skiprows=22, header=None)

# The dataset format:
# - Every 2 rows = 1 observation
# - First row: first 13 predictors
# - Second row: 2 predictors + response (medv)

# Assemble the full predictor matrix
X_data = np.hstack([
    raw_df.values[::2, :],      # Rows 0,2,4,... (first 13 predictors)
    raw_df.values[1::2, :2]     # Rows 1,3,5,... (first two values of second row)
])

# Response vector y (3rd column of second rows)
y = raw_df.values[1::2, 2]

# Add an intercept column
X = sm.add_constant(X_data)

# Number of predictors (+1 for intercept) and observations
p_plus_one = X.shape[1]
n = X.shape[0]

print("Number of predictors (including intercept):", p_plus_one)
print("Number of observations:", n)


Number of predictors (including intercept): 14
Number of observations: 506


Using your function `orthogonal_projection` orthogonally project onto the column space of X by projecting y on each vector of X individually and adding up the projections and call the sum `yhat_naive`.

In [25]:
# Assume y, X, and orthogonal_projection() are already loaded

# Initialize yhat_naive as a zero vector (same shape as y)
yhat_naive = np.zeros_like(y)

# Project y onto each column of X individually and sum
for j in range(X.shape[1]):
    proj = orthogonal_projection(y, X[:, j])
    yhat_naive += proj['a_parallel']

# Inspect
print("First few entries of yhat_naive:")
print(yhat_naive[:5])


First few entries of yhat_naive:
[177.34253286 185.60133556 177.71752148 171.72473472 177.32548791]


How much double counting occurred? Measure the magnitude relative to the true LS orthogonal projection

In [27]:
import statsmodels.api as sm

# 1. Fit the OLS model
model = sm.OLS(y, X).fit()

# 2. Extract the true OLS fitted values
yhat = model.fittedvalues

# 3. Compute the magnitude ratio
mag_ratio = np.sqrt(np.sum(yhat_naive**2)) / np.sqrt(np.sum(yhat**2))

# 4. Print
print("Magnitude ratio (sqrt(sum(yhat_naive^2)) / sqrt(sum(yhat^2))):", mag_ratio)


Magnitude ratio (sqrt(sum(yhat_naive^2)) / sqrt(sum(yhat^2))): 8.997117717661752


Is this ratio expected? Why or why not?


In summary, the ratio is expected to deviate from 1 because the sum of individual (non-orthogonal) projections overestimates the overall projection onto the column space, leading to a larger magnitude for the naive projection compared to the true LS projection.

Convert X into V where V has the same column space as X but has orthogonal columns. You can use the function `orthogonal_projection`. This is the Gram-Schmidt orthogonalization algorithm (part A).

In [31]:
# Initialize V to hold the orthogonalized columns
V = np.empty_like(X)

# The first column of V is simply the first column of X
V[:, 0] = X[:, 0]

# Orthogonalize remaining columns
for j in range(1, X.shape[1]):
    V[:, j] = X[:, j].copy()
    for k in range(j):
        proj = orthogonal_projection(V[:, j], V[:, k])
        V[:, j] = V[:, j] - proj['a_parallel']


Convert V into Q whose columns are the same except normalized. This is the Gram-Schmidt orthogonalization algorithm (part B).

In [34]:
# Create Q by normalizing each column of V
Q = np.empty_like(V)

for j in range(V.shape[1]):
    Q[:, j] = V[:, j] / norm_vec(V[:, j])

# (Optional) If you want to clean memory
del V

# Verify first few columns if you like
print("First few entries of Q:")
print(Q[:5, :5])


First few entries of Q:
[[ 0.04445542 -0.01866158  0.00910601 -0.05766684 -0.00830254]
 [ 0.04445542 -0.01855299 -0.02592754 -0.03907578 -0.01166524]
 [ 0.04445542 -0.0185531  -0.02592756 -0.03907574 -0.01166525]
 [ 0.04445542 -0.01852682 -0.02592218 -0.07931947 -0.00856873]
 [ 0.04445542 -0.01833705 -0.02588335 -0.07939459 -0.00855013]]


Verify Q^T Q is I_{p+1} i.e. Q is an orthonormal matrix.

In [38]:
# Compute QᵀQ
QtQ = Q.T @ Q

# Identity matrix
I_p = np.eye(Q.shape[1])

# Check if QᵀQ ≈ Identity matrix
if np.allclose(QtQ, I_p):
    print("QᵀQ equals the identity matrix, Q is orthonormal.")
else:
    print("QᵀQ does NOT equal the identity matrix!")
    
# Optionally print difference
print("Difference between QᵀQ and Identity:")
print(QtQ - I_p)


QᵀQ equals the identity matrix, Q is orthonormal.
Difference between QᵀQ and Identity:
[[-4.10782519e-15 -2.77555756e-16  6.24500451e-17  2.63677968e-15
  -1.90819582e-17 -3.85802501e-15  7.58074159e-16  2.49800181e-15
   1.67921232e-15 -5.27355937e-16 -6.58847976e-15  4.71844785e-16
  -1.81105131e-15 -1.50573998e-15]
 [-2.77555756e-16 -1.11022302e-16  1.99493200e-17 -6.93889390e-18
  -9.43255890e-18  2.08166817e-17  2.03830008e-17 -5.20417043e-17
   5.72458747e-17  6.93889390e-18 -8.23993651e-17  4.59701721e-17
   1.45283091e-16  3.20923843e-17]
 [ 6.24500451e-17  1.99493200e-17 -2.22044605e-16  1.87350135e-16
   2.08166817e-17  6.24500451e-17 -3.12250226e-17  1.04083409e-16
  -8.67361738e-17  0.00000000e+00 -7.63278329e-17  1.38777878e-16
  -2.08166817e-17 -1.30104261e-18]
 [ 2.63677968e-15 -6.93889390e-18  1.87350135e-16  6.66133815e-16
   2.60208521e-17  1.97758476e-16 -1.18394877e-16  4.85722573e-17
  -6.43473989e-17 -5.55111512e-17  8.32667268e-17  6.93889390e-17
  -5.20417043e-1

Is your Q the same as what results from R's built-in QR-decomposition function?

In [42]:
Q_builtin, R = np.linalg.qr(X)

if np.allclose(np.abs(Q), np.abs(Q_builtin)):
    print("Q matches the built-in QR decomposition Q (up to sign differences).")
else:
    print("Q does not match the built-in QR decomposition Q.")

print("Difference (absolute values):")
print(np.abs(Q) - np.abs(Q_builtin))


Q matches the built-in QR decomposition Q (up to sign differences).
Difference (absolute values):
[[-1.38777878e-17 -1.73472348e-17 -2.77555756e-17 ...  2.13162821e-14
  -3.25000443e-15 -2.90739655e-15]
 [ 0.00000000e+00  1.07552856e-16 -3.46944695e-18 ... -1.17961196e-16
  -5.78963960e-17  2.46330734e-16]
 [ 0.00000000e+00  1.04083409e-17  4.16333634e-17 ... -6.93889390e-17
  -6.11490025e-17  1.52655666e-16]
 ...
 [ 0.00000000e+00  6.93889390e-18  0.00000000e+00 ...  6.93889390e-17
  -8.06646416e-17  2.22044605e-16]
 [ 0.00000000e+00 -6.93889390e-18  0.00000000e+00 ...  0.00000000e+00
  -1.85615412e-16 -4.16333634e-17]
 [ 0.00000000e+00 -6.93889390e-18 -3.46944695e-18 ...  2.77555756e-17
  -1.87350135e-16 -2.77555756e-17]]


Is this expected? Why did this happen?

Yes this is expected because there are an infinite number of orthonormal basis of any column space and the likelihood of them being equal is highly unlikely.  
There are many different orthonormal basis of any column space. 

Project y onto colsp[Q] and verify it is the same as the OLS fit.

In [48]:
# Project y onto colsp[Q]: since Q is orthonormal, projection is Q @ Qᵀ @ y
y_hat_from_Q = Q @ (Q.T @ y)

# Compare to true OLS fitted values
if np.allclose(y_hat_from_Q, yhat):
    print("The projection of y onto colsp[Q] equals the OLS fit.")
else:
    print("There is a discrepancy between the projection and the OLS fit.")

# Optionally, print the small difference
print("Difference (y_hat_from_Q - yhat):")
print(y_hat_from_Q - yhat)


The projection of y onto colsp[Q] equals the OLS fit.
Difference (y_hat_from_Q - yhat):
[-2.77111667e-13  2.27373675e-13  1.42108547e-13  1.77635684e-14
  3.55271368e-14  8.17124146e-14  6.75015599e-14  2.09610107e-13
  1.68753900e-13  1.98951966e-13  2.06057393e-13  1.95399252e-13
 -1.17239551e-13  1.77635684e-14  1.38555833e-13 -3.55271368e-14
 -1.70530257e-13  6.75015599e-14 -1.17239551e-13  3.55271368e-15
  1.15463195e-13  1.13686838e-13  6.92779167e-14  1.24344979e-13
  1.36779477e-13  1.59872116e-13  1.54543045e-13  1.68753900e-13
  1.66977543e-13  1.13686838e-13  9.94759830e-14  2.06057393e-13
  7.10542736e-14  1.29674049e-13  2.06057393e-13  2.84217094e-14
 -7.10542736e-15 -9.94759830e-14 -1.84741111e-13  2.13162821e-14
  0.00000000e+00 -9.59232693e-14 -7.10542736e-14 -1.06581410e-13
  1.03028697e-13  2.84217094e-14 -1.06581410e-14  2.91322522e-13
  2.57571742e-13  1.88293825e-13  1.56319402e-13  2.91322522e-13
  5.32907052e-14  4.61852778e-14 -3.01980663e-13  2.77111667e-13
  

Project y onto colsp[Q] one by one and verify it sums to be the projection onto the whole space.

In [51]:
# Initialize yhat_naive as zeros
yhat_naive = np.zeros_like(y)

# Project y onto each column of Q and sum
for j in range(Q.shape[1]):
    proj = orthogonal_projection(y, Q[:, j])
    yhat_naive += proj['a_parallel']

# Full projection using H = Q @ Qᵀ
H = Q @ Q.T
yhat_full = H @ y

# Compare naive sum vs full projection
if np.allclose(yhat_naive, yhat_full):
    print("The naive projection (one column at a time) equals the full projection (Q @ Qᵀ @ y).")
else:
    print("There is a discrepancy between the projections!")

# Optionally, print difference
print("Difference (yhat_naive - yhat_full):")
print(yhat_naive - yhat_full)


The naive projection (one column at a time) equals the full projection (Q @ Qᵀ @ y).
Difference (yhat_naive - yhat_full):
[ 3.55271368e-15  1.77635684e-14  7.10542736e-15  2.13162821e-14
  1.77635684e-14  1.42108547e-14  2.84217094e-14  2.13162821e-14
  1.59872116e-14  1.42108547e-14  1.42108547e-14  1.06581410e-14
  2.48689958e-14  1.42108547e-14  0.00000000e+00  1.42108547e-14
  7.10542736e-15  1.06581410e-14  7.10542736e-15  1.06581410e-14
  1.24344979e-14  2.13162821e-14  1.42108547e-14  1.59872116e-14
  2.13162821e-14  1.95399252e-14  1.95399252e-14  1.24344979e-14
  7.10542736e-15  1.42108547e-14  1.77635684e-14  1.06581410e-14
  1.24344979e-14  1.42108547e-14  1.24344979e-14  2.48689958e-14
  1.42108547e-14  1.42108547e-14  7.10542736e-15  2.13162821e-14
  7.10542736e-15  1.06581410e-14  1.06581410e-14  1.42108547e-14
  1.77635684e-14  2.13162821e-14  1.06581410e-14  1.06581410e-14
  7.10542736e-15  2.13162821e-14  2.48689958e-14  1.06581410e-14
  3.55271368e-15  3.55271368e-15 

Split the Boston Housing Data into a training set and a test set where the training set is 80% of the observations. Do so at random.

In [54]:
# Assume n, X, and y are already defined. For example:
# n = X.shape[0]
# (Make sure X is a NumPy array and y is a 1D array or similar.)

np.random.seed(123)
n_train = int(np.floor(0.8 * n))
n_test = n - n_train

indices = np.random.permutation(n)
train_indices = indices[:n_train]
test_indices = indices[n_train:]

X_train = X[train_indices, :]
X_test = X[test_indices, :]
y_train = y[train_indices]
y_test = y[test_indices]

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train length:", len(y_train))
print("y_test length:", len(y_test))


X_train shape: (404, 14)
X_test shape: (102, 14)
y_train length: 404
y_test length: 102


Fit an OLS model. Find the s_e in sample and out of sample. Which one is greater? Note: we are now using s_e and not RMSE since RMSE has the n-(p + 1) in the denominator not n-1 which attempts to de-bias the error estimate by inflating the estimate when overfitting in high p. Again, we're just using `sd(e)`, the sample standard deviation of the residuals.

In [57]:
# Fit the OLS model on the training set.
# (Assume X_train and y_train have already been defined,
# and that X_train already includes a constant column if needed.)
model = sm.OLS(y_train, X_train).fit()

# Predict on the test set
y_hat_test = model.predict(X_test)

# Compute in-sample standard error (sample SD of residuals from the training fit)
in_sample_se = np.std(model.resid, ddof=1)

# Compute out-of-sample standard error (sample SD of test residuals)
out_sample_se = np.std(y_test - y_hat_test, ddof=1)

print("In-sample s_e (sample SD of residuals):", in_sample_se)
print("Out-of-sample s_e (sample SD of residuals):", out_sample_se)


In-sample s_e (sample SD of residuals): 4.540082806616528
Out-of-sample s_e (sample SD of residuals): 5.317400950608923


Do these two exercises `Nsim = 1000` times and find the average difference between s_e and ooss_e.

In [60]:
# Assume n, X, and y are already defined.
# Here, n is the total number of observations.
# X is the design matrix and y is the response vector.
# For consistency with the R code (which uses '+0'), we will fit the model without a constant.
# If X already has a constant column, you might need to remove it; here we assume X does NOT include one.

Nsim = 1000
n_train = int(np.floor(0.8 * n))
n_test = n - n_train

s_e_array = np.zeros(Nsim)
oosSSE_array = np.zeros(Nsim)

for i in range(Nsim):
    indices = np.random.permutation(n)
    train_indices = indices[:n_train]
    test_indices = indices[n_train:]

    X_train = X[train_indices, :]
    X_test = X[test_indices, :]
    y_train = y[train_indices]
    y_test = y[test_indices]

    model = sm.OLS(y_train, X_train).fit()

    y_hat_test = model.predict(X_test)

    oosSSE_array[i] = np.std(y_test - y_hat_test, ddof=1)
    s_e_array[i] = np.std(model.resid, ddof=1)

mean_diff = np.mean(s_e_array - oosSSE_array)
print("Average difference (s_e - ooss_e):", mean_diff)


Average difference (s_e - ooss_e): -0.19088290273301492


We'll now add random junk to the data so that `p_plus_one = n_train` and create a new data matrix `X_with_junk.`

In [62]:
# Assume X, n, n_train, and p_plus_one are already defined.
# Create the junk matrix: with n rows and (n_train - p_plus_one) columns.
junk = np.random.randn(n, n_train - p_plus_one)

# Concatenate X and the junk matrix horizontally
X_with_junk = np.hstack((X, junk))

# Print dimensions
print("X shape:", X.shape)
print("X_with_junk shape:", X_with_junk.shape)


X shape: (506, 14)
X_with_junk shape: (506, 404)


Repeat the exercise above measuring the average s_e and ooss_e but this time record these metrics by number of features used. That is, do it for the first column of `X_with_junk` (the intercept column), then do it for the first and second columns, then the first three columns, etc until you do it for all columns of `X_with_junk`. Save these in `s_e_by_p` and `ooss_e_by_p`.

In [ ]:
K = 5  # Test set is 1/5 of the data
n_test = n // K
n_train = n - n_test
Nsim = 100

num_features = X_with_junk.shape[1]

s_e_by_p = np.zeros(num_features)
ooss_e_by_p = np.zeros(num_features)

for j in range(1, num_features + 1):
    oosSSE_array = np.zeros(Nsim)
    s_e_array = np.zeros(Nsim)
    
    for i in range(Nsim):
        all_indices = np.random.permutation(n)
        test_indices = all_indices[:n_test]
        train_indices = all_indices[n_test:]
        
        X_train = X_with_junk[train_indices, :j]
        X_test = X_with_junk[test_indices, :j]
        y_train = y[train_indices]
        y_test = y[test_indices]
        
        model = sm.OLS(y_train, X_train).fit()
        y_hat_test = model.predict(X_test)
        
        oosSSE_array[i] = np.std(y_test - y_hat_test, ddof=1)
        s_e_array[i] = np.std(model.resid, ddof=1)
    
    ooss_e_by_p[j-1] = np.mean(oosSSE_array)
    s_e_by_p[j-1] = np.mean(s_e_array)

print("Average out-of-sample s_e by number of features:")
print(ooss_e_by_p)
print("Average in-sample s_e by number of features:")
print(s_e_by_p)


You can graph them here:

In [ ]:
import pandas as pd
from plotnine import ggplot, aes, geom_line, labs, theme_minimal

# Create data frames for the in-sample and out-of-sample standard errors.
df_in = pd.DataFrame({
    'p': np.arange(1, len(s_e_by_p) + 1),
    's_e': s_e_by_p,
    'series': 'In-sample'
})

df_out = pd.DataFrame({
    'p': np.arange(1, len(ooss_e_by_p) + 1),
    's_e': ooss_e_by_p,
    'series': 'Out-of-sample'
})

# Combine the two dataframes
df = pd.concat([df_in, df_out], ignore_index=True)

# Create the plot using plotnine
plot = (ggplot(df, aes(x='p', y='s_e', color='series'))
        + geom_line()
        + labs(x='Number of features (p)',
               y='s_e',
               title='In-sample vs. Out-of-sample s_e by number of features')
        + theme_minimal()
       )

plot


Is this shape expected? Explain.

Yes this shape is expected because as we add more features the in-sample error will decrease due to the the model fitting the additional features and data. However, the out of sample error will start to get worse due to the over-fitting that is occurring. This will lead to a worse modle that produces worse predictions for data that is out of sample. 

Now repeat the exercise above except use 5-fold CV (K=5 cross validation) for each p. The code below will also plot the oos RMSE. This oos RMSE curve should be similar to the curve in the above problem, but now it will be more stable. 

In [ ]:
# Assume n, y, and X_with_junk are already defined.
# Let p_max be the total number of features (columns) in X_with_junk.
p_max = X_with_junk.shape[1]

K = 5
n_test = n // K

# Create a folds array: assign each observation to one of K folds.
# Replicate 1:K enough times, then shuffle.
folds = np.tile(np.arange(K), n // K + 1)[:n]
np.random.shuffle(folds)

# Initialize a matrix to store the out-of-sample RMSE for each model size and fold.
ooss_e_by_p_k = np.zeros((p_max, K))

for j in range(1, p_max + 1):
    for k in range(K):
        test_indices = np.where(folds == k)[0]
        train_indices = np.where(folds != k)[0]
        
        X_train = X_with_junk[train_indices, :j]
        X_test = X_with_junk[test_indices, :j]
        y_train = y[train_indices]
        y_test = y[test_indices]
        
        model = sm.OLS(y_train, X_train).fit()
        y_hat = model.predict(X_test)
        
        ooss_e_by_p_k[j-1, k] = np.sqrt(np.mean((y_test - y_hat)**2))

# For each model size (row), compute the standard deviation of the RMSE over the K folds.
s_e_by_p = np.std(ooss_e_by_p_k, axis=1)

# Create a DataFrame for plotting
df_plot = pd.DataFrame({
    'p': np.arange(1, p_max + 1),
    's_e': s_e_by_p
})

# Plot using plotnine
plot = (ggplot(df_plot, aes(x='p', y='s_e'))
        + geom_line()
        + labs(x="Number of Features", y="OOS RMSE (sd)", title="5-fold CV OOS RMSE vs. Number of Features")
        + theme_minimal())

plot


Even though the concept of confidence intervals (CIs) will not be on the midterm, construct 95% CIs for each of the oosRMSE measurements by number of features, p. A CI is a real-number interval with a lower bound and upper bound. The formula for the CI is [s_e - 2 * s_s_e, s_e + 2 * s_s_e].

In [ ]:
# Compute the mean out-of-sample RMSE for each model size (row-wise mean)
mean_oos = np.mean(ooss_e_by_p_k, axis=1)

# Compute the sample standard deviation for each model size (across simulations)
sd_oos = np.std(ooss_e_by_p_k, axis=1)

# Construct the 95% confidence intervals
CI_lower = mean_oos - 2 * sd_oos
CI_upper = mean_oos + 2 * sd_oos

# Create a DataFrame with the number of features and the CI bounds.
df_CI = pd.DataFrame({
    'p': np.arange(1, p_max + 1),
    'mean_oos': mean_oos,
    'CI_lower': CI_lower,
    'CI_upper': CI_upper
})

print(df_CI)
